## Step 1: Environment Check

In [ ]:
%%time
import torch
import os

print("="*70)
print("🔧 ENVIRONMENT CHECK")
print("="*70)

print(f"\n📊 PyTorch Version: {torch.__version__}")
print(f"📊 CUDA Available: {torch.cuda.is_available()}")
print(f"📊 GPU Count: {torch.cuda.device_count()}")
print(f"📊 NCCL Available: {torch.cuda.nccl.is_available(torch.cuda.device_count())}")
print(f"📊 cuDNN Version: {torch.backends.cudnn.version()}")

# List GPUs
print("\n📊 GPU Details:")
for i in range(torch.cuda.device_count()):
    props = torch.cuda.get_device_properties(i)
    print(f"   GPU {i}: {props.name}")
    print(f"          Memory: {props.total_memory / 1024**3:.1f} GB")
    print(f"          Compute: {props.major}.{props.minor}")

## Step 2: NCCL Basics - All-Reduce

In [ ]:
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP

print("="*70)
print("🔗 NCCL ALL-REDUCE EXAMPLE")
print("="*70)

def setup_nccl(rank, world_size):
    """Initialize NCCL process group."""
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '29500'
    
    dist.init_process_group(
        backend='nccl',
        rank=rank,
        world_size=world_size
    )

def cleanup():
    """Cleanup NCCL."""
    dist.destroy_process_group()

def all_reduce_demo(rank, world_size):
    """Demonstrate NCCL all-reduce."""
    setup_nccl(rank, world_size)
    
    # Create tensor on each GPU
    torch.cuda.set_device(rank)
    tensor = torch.tensor([rank + 1.0], device=f'cuda:{rank}')
    
    print(f"[GPU {rank}] Before all-reduce: {tensor.item()}")
    
    # All-reduce (sum across GPUs)
    dist.all_reduce(tensor, op=dist.ReduceOp.SUM)
    
    print(f"[GPU {rank}] After all-reduce: {tensor.item()}")
    
    cleanup()

# Note: In Kaggle notebook, run distributed code via subprocess
# This is a conceptual example
print("""
📋 NCCL All-Reduce Pattern:

   Before:    GPU 0: [1.0]     GPU 1: [2.0]
                 \               /
                  \    NCCL    /
                   \  All-Reduce /
                    \         /
                     v       v
   After:     GPU 0: [3.0]     GPU 1: [3.0]

   Result: Sum of all tensors replicated on each GPU
""")

## Step 3: Simple Multi-GPU Tensor Operations

In [ ]:
print("="*70)
print("🔢 MULTI-GPU TENSOR OPERATIONS")
print("="*70)

# Without NCCL - Direct GPU tensor operations
if torch.cuda.device_count() >= 2:
    # Create tensors on different GPUs
    a = torch.randn(1000, 1000, device='cuda:0')
    b = torch.randn(1000, 1000, device='cuda:1')
    
    print(f"\n📊 Tensor A on GPU 0: {a.shape}")
    print(f"📊 Tensor B on GPU 1: {b.shape}")
    
    # Move tensor from GPU 1 to GPU 0
    b_on_gpu0 = b.to('cuda:0')
    print(f"\n📊 B moved to GPU 0")
    
    # Now we can do operations
    c = a @ b_on_gpu0
    print(f"📊 Result C = A @ B: {c.shape}")
    
    # Memory status
    for i in range(2):
        mem = torch.cuda.memory_allocated(i) / 1024**2
        print(f"📊 GPU {i} memory used: {mem:.1f} MB")
    
    # Cleanup
    del a, b, b_on_gpu0, c
    torch.cuda.empty_cache()
    print("\n✅ Tensors cleaned up")
else:
    print("⚠️ Need 2 GPUs for this demo")

## Step 4: Data Parallel Training Pattern

In [ ]:
import torch.nn as nn

print("="*70)
print("📊 DATA PARALLEL TRAINING")
print("="*70)

# Simple model
class SimpleModel(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=2048, output_dim=512):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
    
    def forward(self, x):
        return self.layers(x)

# Create model and wrap with DataParallel
model = SimpleModel().cuda()

if torch.cuda.device_count() >= 2:
    # Use both GPUs
    model = nn.DataParallel(model, device_ids=[0, 1])
    print("\n✅ Model wrapped with DataParallel (GPU 0, 1)")
else:
    print("\n⚠️ Single GPU mode")

print(f"📊 Model parameters: {sum(p.numel() for p in model.parameters()) / 1e6:.2f}M")

# Test forward pass
batch = torch.randn(64, 1024).cuda()
output = model(batch)
print(f"📊 Input: {batch.shape} → Output: {output.shape}")

# Cleanup
del model, batch, output
torch.cuda.empty_cache()

## Step 5: DistributedDataParallel Script Pattern

In [ ]:
print("="*70)
print("🔧 DDP TRAINING SCRIPT PATTERN")
print("="*70)

ddp_script = '''
#!/usr/bin/env python3
"""DDP Training Script for Kaggle Dual T4"""

import os
import torch
import torch.nn as nn
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, DistributedSampler
from torch.utils.data import TensorDataset

def setup(rank, world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '29500'
    dist.init_process_group("nccl", rank=rank, world_size=world_size)
    torch.cuda.set_device(rank)

def cleanup():
    dist.destroy_process_group()

def train(rank, world_size):
    setup(rank, world_size)
    
    # Model
    model = nn.Linear(1024, 512).cuda(rank)
    model = DDP(model, device_ids=[rank])
    
    # Fake dataset
    dataset = TensorDataset(
        torch.randn(1000, 1024),
        torch.randn(1000, 512)
    )
    
    # Distributed sampler
    sampler = DistributedSampler(dataset, num_replicas=world_size, rank=rank)
    loader = DataLoader(dataset, batch_size=32, sampler=sampler)
    
    # Training loop
    optimizer = torch.optim.Adam(model.parameters())
    criterion = nn.MSELoss()
    
    for epoch in range(3):
        sampler.set_epoch(epoch)  # Important!
        for x, y in loader:
            x, y = x.cuda(rank), y.cuda(rank)
            optimizer.zero_grad()
            loss = criterion(model(x), y)
            loss.backward()  # NCCL syncs gradients here
            optimizer.step()
        
        if rank == 0:
            print(f"Epoch {epoch+1} complete")
    
    cleanup()

if __name__ == "__main__":
    world_size = torch.cuda.device_count()
    torch.multiprocessing.spawn(train, args=(world_size,), nprocs=world_size)
'''

# Save script
with open('/kaggle/working/ddp_train.py', 'w') as f:
    f.write(ddp_script)

print("\n📝 DDP training script saved to /kaggle/working/ddp_train.py")
print("\n📋 Key DDP Concepts:")
print("   • DistributedSampler splits data across GPUs")
print("   • DDP automatically syncs gradients via NCCL")
print("   • sampler.set_epoch() for proper shuffling")
print("   • Each GPU processes batch_size samples")

## Step 6: Run DDP Script

In [ ]:
%%time
print("="*70)
print("🏃 RUNNING DDP TRAINING")
print("="*70)

!python /kaggle/working/ddp_train.py

print("\n✅ DDP training complete!")

## Step 7: NCCL with llcuda Inference

In [ ]:
print("="*70)
print("🔌 COMBINING DDP TRAINING + LLCUDA INFERENCE")
print("="*70)

print("""
📋 Architecture Pattern:

┌─────────────────────────────────────────────────────────────────────┐
│                    TRAINING + INFERENCE PIPELINE                    │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   PHASE 1: Fine-tuning with DDP                                    │
│   ┌─────────────────┐      ┌─────────────────┐                     │
│   │   GPU 0 (T4)    │◄────►│   GPU 1 (T4)    │   NCCL             │
│   │   Model Shard   │      │   Model Shard   │   All-Reduce       │
│   └────────┬────────┘      └────────┬────────┘                     │
│            │                        │                               │
│            └────────┬───────────────┘                               │
│                     ▼                                               │
│             [Fine-tuned Model]                                      │
│                     │                                               │
│                     ▼                                               │
│   PHASE 2: Export to GGUF                                          │
│   ┌───────────────────────────────────────┐                        │
│   │   Unsloth merge + GGUF quantization   │                        │
│   │   → model.Q4_K_M.gguf                 │                        │
│   └───────────────────┬───────────────────┘                        │
│                       ▼                                             │
│   PHASE 3: Inference with llcuda                                   │
│   ┌─────────────────┐      ┌─────────────────┐                     │
│   │   GPU 0 (T4)    │◄────►│   GPU 1 (T4)    │   tensor-split     │
│   │   llama-server  │      │   llama-server  │   (native CUDA)    │
│   │   Layers 0-15   │      │   Layers 16-31  │                     │
│   └─────────────────┘      └─────────────────┘                     │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘

📝 Notes:
• Training: Uses NCCL for gradient synchronization
• Inference: Uses native CUDA tensor-split (NOT NCCL)
• Memory: Can run both phases with proper cleanup
""")

## Step 8: NCCL Operations Reference

In [ ]:
print("="*70)
print("📚 NCCL OPERATIONS REFERENCE")
print("="*70)

print("""
🔹 Collective Operations:

┌──────────────────┬─────────────────────────────────────────────────┐
│ Operation        │ Description                                      │
├──────────────────┼─────────────────────────────────────────────────┤
│ all_reduce       │ Reduce + Broadcast (sum/avg across all GPUs)    │
│ all_gather       │ Gather tensors from all GPUs to all GPUs        │
│ reduce           │ Reduce to single GPU                            │
│ broadcast        │ Send from one GPU to all GPUs                   │
│ reduce_scatter   │ Reduce + Scatter result                         │
│ all_to_all       │ Full exchange between all GPUs                  │
└──────────────────┴─────────────────────────────────────────────────┘

🔹 Reduce Operations:

   dist.ReduceOp.SUM      - Sum values
   dist.ReduceOp.PRODUCT  - Multiply values
   dist.ReduceOp.MIN      - Minimum value
   dist.ReduceOp.MAX      - Maximum value
   dist.ReduceOp.AVG      - Average values

🔹 Common Patterns:

   # Gradient sync (DDP does this automatically)
   dist.all_reduce(gradient, op=dist.ReduceOp.SUM)
   gradient /= world_size
   
   # Gather outputs from all GPUs
   gathered = [torch.zeros_like(tensor) for _ in range(world_size)]
   dist.all_gather(gathered, tensor)
   
   # Broadcast from rank 0
   dist.broadcast(tensor, src=0)
""")

## Step 9: Memory Management for DDP + Inference

In [ ]:
import gc

print("="*70)
print("💾 MEMORY MANAGEMENT")
print("="*70)

def cleanup_gpu_memory():
    """Clean up GPU memory after training before inference."""
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    
    print("\n📊 GPU Memory After Cleanup:")
    for i in range(torch.cuda.device_count()):
        allocated = torch.cuda.memory_allocated(i) / 1024**2
        reserved = torch.cuda.memory_reserved(i) / 1024**2
        print(f"   GPU {i}: {allocated:.1f} MB allocated, {reserved:.1f} MB reserved")

cleanup_gpu_memory()

print("""
📋 Memory Strategy:

1. After DDP Training:
   • Call dist.destroy_process_group()
   • Delete model, optimizer, data loaders
   • Call cleanup_gpu_memory()

2. Before llcuda Inference:
   • Ensure training cleanup is complete
   • Start llama-server with appropriate model
   • Use --tensor-split for multi-GPU

3. T4 Memory Budget:
   • Total: 15GB per GPU (30GB total)
   • DDP Training: ~10-12GB per GPU
   • Inference: Q4_K_M 7B fits on single T4
   • Inference: 70B IQ3_XS needs both T4s
""")

## Step 10: Full Pipeline Example

In [ ]:
print("="*70)
print("🔄 FULL PIPELINE: TRAIN → EXPORT → INFERENCE")
print("="*70)

full_pipeline = '''
# Full Pipeline Script

# ═══════════════════════════════════════════════════════════
# PHASE 1: DDP Training with NCCL
# ═══════════════════════════════════════════════════════════

# Run DDP training script
!torchrun --nproc_per_node=2 train_ddp.py \
    --model_name "unsloth/Qwen2.5-1.5B" \
    --output_dir "./fine_tuned_model" \
    --epochs 3

# Cleanup GPU memory after training
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

# ═══════════════════════════════════════════════════════════
# PHASE 2: Merge & Export to GGUF
# ═══════════════════════════════════════════════════════════

from unsloth import FastLanguageModel

# Load and merge LoRA
model, tokenizer = FastLanguageModel.from_pretrained("./fine_tuned_model")
model.save_pretrained_merged("./merged_model", tokenizer)

# Export to GGUF
model.save_pretrained_gguf(
    "./gguf_model",
    tokenizer,
    quantization_method="q4_k_m"
)

del model, tokenizer
gc.collect()
torch.cuda.empty_cache()

# ═══════════════════════════════════════════════════════════
# PHASE 3: Inference with llcuda
# ═══════════════════════════════════════════════════════════

from llcuda.server import ServerManager, ServerConfig

config = ServerConfig(
    model_path="./gguf_model/model-Q4_K_M.gguf",
    host="127.0.0.1",
    port=8080,
    n_gpu_layers=99,
    tensor_split="0.5,0.5",  # Dual T4
    flash_attn=True,
)

server = ServerManager()
server.start_with_config(config)
server.wait_until_ready()

# Now use OpenAI API at http://127.0.0.1:8080
'''

print(full_pipeline)

## Step 11: Cleanup

In [ ]:
print("🧹 Final Cleanup")

# Clean up test script
import os
if os.path.exists('/kaggle/working/ddp_train.py'):
    os.remove('/kaggle/working/ddp_train.py')
    print("✅ Removed test script")

# Clear GPU memory
import gc
gc.collect()
torch.cuda.empty_cache()
print("✅ GPU memory cleared")

## 📚 Summary

### NCCL vs llama.cpp Multi-GPU:

| Feature | NCCL | llama.cpp |
|---------|------|-----------|
| Purpose | Training, gradient sync | Inference |
| Protocol | All-reduce, broadcast | Tensor-split |
| Integration | PyTorch DDP | Native CUDA |
| Use Case | Fine-tuning | Serving |

### Key Takeaways:
1. **NCCL** is for PyTorch distributed training
2. **llama.cpp** uses native CUDA for multi-GPU inference
3. Both can run on Kaggle's dual T4 GPUs
4. Clean GPU memory between training and inference

### Pipeline Pattern:
```
DDP Training (NCCL) → GGUF Export → llcuda Inference (tensor-split)
```

---

**Next:** [09-large-models-kaggle](09-large-models-kaggle-llcuda-v2.2.0.ipynb)